## Merge dataset to a week

### Import libaraies

In [1]:
import datetime
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import folium
import random

### Help functions

In [2]:
# Find the day according to the filename(date)
def day_calculator(s):
    s = s.split(".")[0]
    year, month, date = s.split("-")
    res = datetime.datetime(int(year), int(month), int(date))
    return res.strftime("%a")

In [3]:
# Implement the merge step and export as csv file
def merge_missing(df, df_new, date):
    missing_rows = df_new[~df_new['uid'].isin(df['uid'])]
    file_name = date + ".csv"
    if len(missing_rows) == 0:
        return False, df
    else:
        df_update = pd.concat([df, missing_rows])
        df_update.to_csv(file_name, index = False)
        return True, df_update

In [4]:
# Find the number of users after merge and their uid
def find_merge_amount(dict_merge, date):
    dict_mon = dict_merge[date].copy()

    df_mon1 = pd.DataFrame(columns=['uid', 't', 'lat', 'long'])
    df_mon2 = pd.DataFrame(columns=['uid', 't', 'lat', 'long'])
    full = 0
    idx = 0
    repeated = dict()
    while len(dict_mon) > 0: 
        if len(df_mon1) < 100 * 48:   
            file = dict_mon[idx]
            df = pd.read_csv('data/' + file)
            flag, df_mon1 = merge_missing(df_mon1, df, date)
            dict_mon.remove(file)
            users = df['uid'].unique()
            for user in users:
                if user not in repeated.keys():
                    repeated[user] = 1
                else:
                    repeated[user] += 1
    
    print(date, ":", len(repeated))
    
    contained_users = list(repeated.keys())

    return contained_users

In [5]:
# Find the common users for a week
def find_occurrence(arr):
    count = 0
    dict_count = dict()
    for ele in arr:
        if ele not in dict_count.keys():
            dict_count[ele] = 1
        else:
            dict_count[ele] += 1
            
    for user in dict_count.keys():
        if dict_count[user] >= 7:
            count += 1
    return count

### Merge

Read and find the exact day for each file

In [6]:
filenames = os.listdir(r'data')
print(filenames)
dates = list(map(day_calculator, filenames))
print(dates)

['2008-05-30.csv', '2008-10-06.csv', '2008-10-25.csv', '2008-10-26.csv', '2008-10-28.csv', '2008-10-29.csv', '2008-10-30.csv', '2008-10-31.csv', '2008-11-01.csv', '2008-11-02.csv', '2008-11-03.csv', '2008-11-04.csv', '2008-11-05.csv', '2008-11-06.csv', '2008-11-07.csv', '2008-11-08.csv', '2008-11-09.csv', '2008-11-10.csv', '2008-11-12.csv', '2008-11-13.csv', '2008-11-14.csv', '2008-11-15.csv', '2008-11-16.csv', '2008-11-18.csv', '2008-11-21.csv', '2008-12-03.csv', '2008-12-04.csv', '2008-12-05.csv', '2008-12-06.csv', '2008-12-07.csv', '2008-12-09.csv', '2008-12-10.csv', '2008-12-11.csv', '2008-12-12.csv', '2008-12-13.csv', '2008-12-14.csv', '2008-12-19.csv', '2009-01-09.csv', '2009-01-16.csv', '2009-02-07.csv', '2009-02-10.csv', '2009-02-13.csv', '2009-02-14.csv', '2009-02-15.csv', '2009-02-16.csv', '2009-02-17.csv', '2009-02-18.csv', '2009-02-19.csv', '2009-02-20.csv', '2009-02-21.csv', '2009-02-22.csv', '2009-02-24.csv', '2009-02-25.csv', '2009-02-26.csv', '2009-03-05.csv', '2009-03-

Find the number of each day and combine them by a dictionary

In [7]:
all_dates = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

for day in all_dates:
    count_day = dates.count(day)
    print(day, "=", count_day)
    
dict_merge = {}
for day in all_dates:
    dict_merge[day] = []
    for i in range(len(filenames)):
        if dates[i] == day:
            dict_merge[day].append(filenames[i])

Mon = 10
Tue = 15
Wed = 12
Thu = 16
Fri = 20
Sat = 19
Sun = 16


In [8]:
# Check user number
all_users = []

for date in all_dates:
    contained_users = find_merge_amount(dict_merge, date)

    if all_users == []:
        all_users = contained_users
    else:
        all_users = all_users + contained_users

# print(all_users)
print("overall:", find_occurrence(all_users))

Mon : 49
Tue : 47
Wed : 45
Thu : 50
Fri : 62
Sat : 58
Sun : 50
overall: 32


### Switch time and concatenate with the next day 

In [9]:
def switch_time(df_prev, df_next):
    user_prev = sorted(list(df_prev['uid'].drop_duplicates()))
    user_next = sorted(list(df_next['uid'].drop_duplicates()))
    users = []
    
    for user in user_prev:
        if user in user_next:
            users.append(user)
            
    df_update = pd.DataFrame(columns=['uid', 't', 'lat', 'long'])
    
    for user in users:
        prev_day = df_prev[(df_prev['uid'] == user) & (df_prev['t'] >= 12) & (df_prev['t'] < 48)]
        next_morning = df_next[(df_next['uid'] == user) & (df_next['t'] >= 0) & (df_next['t'] < 12)]
        df_update = pd.concat([df_update, prev_day])
        df_update = pd.concat([df_update, next_morning])
        
    print("user amount:", len(users))
    
    return df_update

In [10]:
filepath = 'data_by_day/'
all_dates = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# import datasets
df_Mon = pd.read_csv(filepath + 'Mon.csv')
df_Tue = pd.read_csv(filepath + 'Tue.csv')
df_Wed = pd.read_csv(filepath + 'Wed.csv')
df_Thu = pd.read_csv(filepath + 'Thu.csv')
df_Fri = pd.read_csv(filepath + 'Fri.csv')
df_Sat = pd.read_csv(filepath + 'Sat.csv')
df_Sun = pd.read_csv(filepath + 'Sun.csv')

In [11]:
df_Mon_update = switch_time(df_Sun, df_Mon)
df_Tue_update = switch_time(df_Mon, df_Tue)
df_Wed_update = switch_time(df_Tue, df_Wed)
df_Thu_update = switch_time(df_Wed, df_Thu)
df_Fri_update = switch_time(df_Thu, df_Fri)
df_Sat_update = switch_time(df_Fri, df_Sat)
df_Sun_update = switch_time(df_Sat, df_Sun)

user amount: 39
user amount: 38
user amount: 41
user amount: 42
user amount: 44
user amount: 52
user amount: 48


In [12]:
df_Mon_update.to_csv("data_by_day/Mon_update.csv", index = False)
df_Tue_update.to_csv("data_by_day/Tue_update.csv", index = False)
df_Wed_update.to_csv("data_by_day/Wed_update.csv", index = False)
df_Thu_update.to_csv("data_by_day/Thu_update.csv", index = False)
df_Fri_update.to_csv("data_by_day/Fri_update.csv", index = False)
df_Sat_update.to_csv("data_by_day/Sat_update.csv", index = False)
df_Sun_update.to_csv("data_by_day/Sun_update.csv", index = False)

### Visualizations

In [13]:
def create_random(upper, num):
    res = set()
    while len(res) < num:
        n = random.randint(0, upper)
        res.add(n)
    return res

def plot_dates(trajectory, date):
    m = folium.Map(location=[39.9, 116.4], zoom_start=9, tiles="cartodb positron")
    colors = ['gray', 'black', 'lightred', 'cadetblue', 'red', 'darkblue', 'green', 'lightblue', 'beige', 'blue', 'lightgreen', 'white', 'orange', 'darkgreen', 'purple', 'darkred', 'pink', 'darkpurple', 'lightgray']
    save_file_name = date + ".html"
    
    index = create_random(len(trajectory) - 1, 5)
    
    for i in index:
        for location in range(len(trajectory[i])):
            popup = f"{location+1}"
            loc = f"{location+1}"
            folium.Marker(location=trajectory[i][location],
                         tooltip=popup,
                         icon=folium.Icon(icon=loc, prefix='fa', color=colors[i])
                         ).add_to(m)
            
        folium.PolyLine(trajectory[i], color="red", weight=2.5, opacity=1).add_to(m)
    m.save(save_file_name)

In [14]:
trajectory = []
daily_trajectory = []
current_date = dict_merge['Mon']

for ele in current_date:
    df = pd.read_csv('data/'+ele)
    
    for i, row in df.iterrows():
        if row['uid'] == 3:
            daily_trajectory.append((row['lat'], row['long']))
    trajectory.append(daily_trajectory)

In [15]:
plot_dates(trajectory, "Mon")